In [1]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
# SQL Alchemy
from sqlalchemy import create_engine, event
import time
from datetime import datetime

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name
from config import remote_db_user, remote_db_pwd, remote_db_endpoint, remote_db_port, remote_db_name

In [2]:
# Create Engine and Pass in MySQL Connection

# Remote MySQL DB Connection  
remote_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
remote_conn = remote_engine.connect()

# Create remote database
try:
    remote_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {remote_db_name}")
except Exception as e:
    print(e)

# Connect to remote database
try:
    remote_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
    remote_conn = remote_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
remote_engine.table_names()

(pymysql.err.ProgrammingError) (1007, "Can't create database 'fantasy_project'; database exists")
[SQL: CREATE DATABASE fantasy_project]
(Background on this error at: http://sqlalche.me/e/13/f405)


['fantasy_stats', 'super_bowl_table']

In [3]:
def load_rows():
    # load 300 rows view
    load_300 = browser.links.find_by_partial_text('300')
    load_300.click()
    

    
    # load more rows
    #load_more = browser.links.find_by_partial_text('Load More')
    #load_more[1].click()
    # for x in range(6):
    #     load_more[1].click()

In [4]:
def stats():
    # save the stats table as a df

    try:
        stats_table = fantasy_soup.find("table", id="stats_grid")
        stats_df = pd.read_html(f'<table>${stats_table.tbody}</table>')[0]
    except Exception as e:
        print('---- exception in stats() function ----')
        print(e)
        if hasattr(e, 'message'):
            print(e.message)
        print(stats_table)
        print(f'tbody: {stats_table}')
        print('--------')
        print(fantasy_soup)
        return
    return stats_df

In [5]:
def names():
    
    # save the names table as a df
    try:
        table_names = fantasy_soup.find("div", class_="k-grid-content-locked")
        #if table_names.tbody == '':
        names_df = pd.read_html(f'<table>${table_names.tbody}</table>')[0]
#         else:
#             print('null table... bypassing')
#             raise Exception       
        
    except Exception as e:
        print('---- exception in names() function ----')
        print(e)
        if hasattr(e, 'message'):
            print(e.message)
        print(table_names)
        print(f'tbody: {table_names.tbody}')
        
        print(fantasy_soup)
        print('--------')
        return
        
    return names_df

In [6]:
def combine(season, names_df, stats_df):
    
    try:
        col_names = ['RK', 'NAME', 'TEAM', 'POS', 'GMS', 'PASS YDS', 'PASS TD', 'PASS INT', 'RUSH YDS', 'RUSH TD', 'REC', 'REC YDS', 'REC TD', 'DEF SCK', 'DEF INT', 'DEF FF', 'DEF FR', 'FPTS/G', 'FPTS']
        combined_df = pd.concat([names_df, stats_df], axis=1)
        combined_df.columns = col_names

        combined_df['SEASON'] = season
        combined_df_reordered = combined_df
        combined_df_reordered = combined_df.reindex(columns=['SEASON', 'RK', 'NAME', 'TEAM', 'POS', 'GMS', 'PASS YDS', 'PASS TD', 'PASS INT', 'RUSH YDS', 'RUSH TD', 'REC', 'REC YDS', 'REC TD', 'DEF SCK', 'DEF INT', 'DEF FF', 'DEF FR', 'FPTS/G', 'FPTS'])
    except Exception as e:
        print('*********************')
        print('exception in combine()')
        print(combined_df.columns)
        print(e)
        if hasattr(e, 'message'):
            print(e.message)
        print(fantasy_soup)
        return

    print(f'done combining {season}')
    return combined_df_reordered
    

In [9]:
def to_db(combined_df):
    
    @event.listens_for(remote_engine, 'before_cursor_execute')
    def receive_before_cursor_execute(remote_engine, cursor, statement, params, context, executemany):
        if executemany:
            cursor.fast_executemany = True
            
    #upload dataframe to database
    combined_df.to_sql(name='fantasy_stats', if_exists='append', con=remote_conn, index=False)
    

In [14]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Steven\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [15]:
# URL of page to be scraped

for season in range(2002, 2021):
    print(season)
    print('---')
    url = (f'https://fantasydata.com/nfl/fantasy-football-leaders?season={season}&seasontype=1&scope=1&subscope=1&startweek=1&endweek=1&aggregatescope=1&range=1')
    url

    # Retrieve page with the browser module
    browser.visit(url)
    time.sleep(10)

    # setup html parser 
    # TO DO: set this up to be global
    html = browser.html
    fantasy_soup = BeautifulSoup(html, 'html.parser')
    
    load_rows()
    
    # setup html parser 
    # TO DO: set this up to be global
    
    time.sleep(10)
    html = browser.html
    fantasy_soup = BeautifulSoup(html, 'html.parser')
    
    names_df = names()
    stats_df = stats()
    combined_df = combine(season, names_df, stats_df)
    to_db(combined_df)

browser.quit()

combined_df.head()

2002
---
done combining 2002
2003
---
done combining 2003
2004
---
done combining 2004
2005
---
done combining 2005
2006
---
done combining 2006
2007
---
done combining 2007
2008
---
done combining 2008
2009
---
done combining 2009
2010
---
done combining 2010
2011
---
done combining 2011
2012
---
done combining 2012
2013
---
done combining 2013
2014
---
done combining 2014
2015
---
done combining 2015
2016
---
done combining 2016
2017
---
done combining 2017
2018
---
done combining 2018
2019
---
done combining 2019
2020
---
done combining 2020


,SEASON,RK,NAME,TEAM,POS,GMS,PASS YDS,PASS TD,PASS INT,RUSH YDS,RUSH TD,REC,REC YDS,REC TD,DEF SCK,DEF INT,DEF FF,DEF FR,FPTS/G,FPTS
0,2020,1,Josh Allen,BUF,QB,16,4544,37,10,421,8,1,12,1,0.0,0,0,0,24.7,395.06
1,2020,2,Aaron Rodgers,GB,QB,16,4299,48,5,149,3,1,-6,0,0.0,0,0,0,23.9,382.26
2,2020,3,Kyler Murray,ARI,QB,16,3971,26,12,819,11,0,0,0,0.0,0,0,0,23.7,378.74
3,2020,4,Patrick Mahomes,KC,QB,15,4740,38,6,308,2,0,0,0,0.0,0,0,0,25.0,374.40
4,2020,5,Deshaun Watson,HOU,QB,16,4823,33,7,444,3,0,0,0,0.0,0,0,0,23.1,369.32
